In [ ]:
# 출처 : https://github.com/netease-youdao/BCEmbedding?tab=readme-ov-file
# bce-embedding 모델과 bce-relank 를 이용하여 langchain으로 RAG 구현 하는 예제 

#!pip install BCEmbedding==0.1.5
#!pip install pypdf

#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [1]:
# We provide the advanced preproc tokenization for reranking.
from BCEmbedding.tools.langchain import BCERerank

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.retrievers import ContextualCompressionRetriever

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# 임베딩 모델 로딩
embedding_model_path = './bce-embedding-base_v1'
embedding_model_kwargs = {'device': 'cpu'} # {'device': 'cuda:0'}
#embedding_encode_kwargs = {'batch_size': 32, 'normalize_embeddings': True, 'show_progress_bar': False}
embedding_encode_kwargs = {'batch_size': 32, 'normalize_embeddings': True}

embed_model = HuggingFaceEmbeddings(
  model_name=embedding_model_path,
  model_kwargs=embedding_model_kwargs,
  encode_kwargs=embedding_encode_kwargs
)

05/20/2024 08:26:50 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: ./bce-embedding-base_v1
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
embed_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='./bce-embedding-base_v1', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'batch_size': 32, 'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [4]:
# rerank 모델 로딩 
# => top_n : 6개
reranker_model_path = './bce-reranker-base_v1'
reranker_args = {'model': reranker_model_path, 'top_n': 6, 'device': 'cpu'}
reranker = BCERerank(**reranker_args)


05/20/2024 08:27:08 - [INFO] -BCEmbedding.models.RerankerModel->>>    Loading from `./bce-reranker-base_v1`.
05/20/2024 08:27:08 - [INFO] -BCEmbedding.models.RerankerModel->>>    Execute device: cpu;	 gpu num: 0;	 use fp16: False


In [5]:
reranker

BCERerank(client='BCEmbedding.models.RerankerModel', top_n=6, model='./bce-reranker-base_v1')

In [6]:
# pdf 파일 불러와서 splitter 만듬.

documents = PyPDFLoader("../doc_test/25.출장여비규정_16.04.01.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [7]:
import time
start_time = time.time()

# example 1. retrieval with embedding and reranker
retriever = FAISS.from_documents(texts, embed_model, 
                                 distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT).as_retriever(search_type="similarity", search_kwargs={"score_threshold": 0.4, "k": 10})

# 문맥압축검색기 적용
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=retriever
)

end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')


05/20/2024 08:27:35 - [INFO] -faiss.loader->>>    Loading faiss with AVX2 support.
05/20/2024 08:27:35 - [INFO] -faiss.loader->>>    Successfully loaded faiss with AVX2 support.


*time:9.19


In [8]:
import time
start_time = time.time()

query='국내출장시 1박 숙박비는 얼마까지 사용가능한가요?'

response = compression_retriever.get_relevant_documents(query)

end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


*time:2.54


In [9]:
response

[Document(page_content='출장여비                                                                            6/9  \n별표1                     국내출장여비   \n                                                             \n \n(단위 : 원) \n구 분 철도운임  선박운임  항공운임  자동차운임  일비 \n(1일당) 숙박비 \n(1박당) 식비 \n(1일당) \n임원이상  실비 \n(KTX특실) 실비 \n(1등급) 실비 \n(일반실) 실비 20,000 실비 25,000 \n부장이하  실비 \n(KTX일반) 실비 \n(2등급) 실비 \n(일반실) 실비 20,000 실비 \n(상한액 ㅣ \n서울특별시  \n70,000원, \n광역시 \n60,000원, \n그 밖의 지\n역 50,000\n원) 20,000 \n주1. 임원은 일비와 식비를 실비로 정산할 수 있다.', metadata={'source': '../doc_test/25.출장여비규정_16.04.01.pdf', 'page': 5, 'relevance_score': 0.6011719703674316}),
 Document(page_content='출장여비                                                                            7/9 라 실비(상한액 81) 37 \n차장 이하 가 26 실비(상한액 155) 67 \n나 실비(상한액 123) 49 \n다 실비(상한액 90) 37 \n라 실비(상한액 77) 30 \n주 1. 귀국일 및 기내박의  경우, ‘라’ 기준의 일비와 식비만 지급한다 . \n주 2. 대표이사는  일비, 숙박비, 식비를 실비로 할 수 있다. \n주 3. 국가 및 도시별 등급 구분은 다음과 같다. \n가. 가등급: 도쿄, 뉴욕, 런던, 로스엔젤레스 , 모스크바 , 샌프란시스코 , 워싱턴 D

In [10]:
from langchain import hub
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [13]:
from langchain_openai import ChatOpenAI
# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)
    
# 단계 8: 체인 생성(Create Chain)
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
import time
start_time = time.time()

# 쿼리 해봄.
question = "국내출장시 1박 숙박비는 얼마까지 사용가능한가요?"
response = rag_chain.invoke(question)

print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')

05/20/2024 08:28:54 - [INFO] -httpx->>>    HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[HUMAN]
국내출장시 1박 숙박비는 얼마까지 사용가능한가요?

[AI]
국내출장시 1박 숙박비는 서울특별시 70,000원, 광역시 60,000원, 그 밖의 지역 50,000원까지 사용 가능합니다. 임원 이상은 일비와 식비를 실비로 정산할 수 있습니다. 대표이사는 일비, 숙박비, 식비를 실비로 할 수 있습니다.
*time:5.09


In [16]:
import time
start_time = time.time()

# 쿼리 해봄.
question = "해외 미국 출장시  1박 숙박비는 얼마까지 사용가능한가요?"
response = rag_chain.invoke(question)

print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

end_time = time.time()
formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'*time:{formatted_elapsed_time}')

05/20/2024 08:30:05 - [INFO] -httpx->>>    HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[HUMAN]
해외 미국 출장시  1박 숙박비는 얼마까지 사용가능한가요?

[AI]
해외 미국 출장시 1박 숙박비는 실비 중에서 상한액이 155인 경우 67만원까지 사용 가능합니다. 대표이사는 일비, 숙박비, 식비를 실비로 할 수 있으며, 국가 및 도시별 등급에 따라 숙박비가 정해집니다.
*time:5.00
